# Database Relationships Revisited

- many-to-many relationship

- Querying a many-to-many relationship is a two-step process. To obtain the list of classes a student is taking, you start from the one-to-many relationship between students and registrations and get the list of registrations for the desired student. Then the one-tomany relationship between classes and registrations is traversed in the many-to-one direction to obtain all the classes associated with the registrations retrieved for the student. Likewise, to find all the students in a class, you start from the class and get a list of registrations, then get the students linked to those registrations.

## self - Referental Relationships

- should care the db relationship details
- like backref, joined, lazy, dynamic, cascade//

## Query Followed Posts Using a Database Join

In [ ]:
db.session.query(Post).select_from(Follow).\
    filter_by(follower_id=self.id).\
    join(Post, Follow.followed_id == Post.author_id)

- All the queries that you have seen so far start from the query attribute of the model that is queried. That format does not work well for this query, because the query needs to return posts rows, yet the first operation that needs to be done is to apply a filter to the follows table. So a more basic form of the query is used instead. To fully understand this query, each part should be looked at individually

- db.session.query(Post) specifies that this is going to be a query that returns Post objects
- select_from(Follow) says that the query begins with the Follow model
- filter_by(follower_id=self.id) performs the filtering of the follows table by the follower user.
- join(Post, Follow.followed_id == Post.author_id) joins the results of filter_by() with the Post objects

In [ ]:
Post.query.join(Follow, Follow.followed_id == Post.author_id)\
    .filter(Follow.follower_id == self.id)

# User Comments

![main](../comment_database.png "main")

## Comment MOderation

# Application Programming Interfaces

- In recent years, there has been a trend in web applications to move more and more of the business logic to the client side, producing an architecture that is known as Rich Internet Application (RIA). In RIAs, the server’s main (and sometimes only) function is to provide the client application with data retrieval and storage services. In this model, the server becomes a web service or Application Programming Interface (API).

- There are several protocols by which RIAs can communicate with a web service. Remote Procedure Call (RPC) protocols such as XML-RPC or its derivative Simplified Object Access Protocol (SOAP) were popular choices a few years ago. More recently, the Representational State Transfer (REST) architecture has emerged as the favorite for web applications due to it being built on the familiar model of the World Wide Web.

## Intro to REST
- Representational State Transfer

> Six-Defining Characteristics

- Client Server : There must be a clear separation between the clients and the server
- Stateless : A client request must contain all the information that is necessary to carry it out. The server must not store any state about the client that persists from one request to the next.
- Cache : Responses from the server can be labeled as cacheable or non cacheable so that clinets (or intermediaries between clients and servers) can use a cache for optimization purposes.
- Uniform Interface : the protocol by which clients access server resources must be consistent, well defined, and standardized. The commonly used uniform interface of REST web services is the HTTP protocol.
- Layered System : Proxy servers, caches, or gateways can be inserted between clients and servers as necessary to improve performance, reliability, adn scalability.
- Code-on-Demand : Clients can optionally download code from the server to execute their context.

### Request Methods

### Request and Response body

- The two formats commonly used with RESTful web services are JavaScript Object Notation (JSON) and Extensible Markup Language (XML). For web-based RIAs, JSON is attractive because of its close ties to JavaScript, the client-side scripting language used by web browsers.

### Versioning

- For these reasons, web services need to be more tolerant than regular web applications and be able to work with old versions of its clients. A common way to address this problem is to version the URLs handled by the web service. For example, the first release of the blogging web service could expose the collection of blog posts at /api/v1.0/posts/. 

### RESTful Web Services with Flask

- Flask makes it very easy to create RESTful web services. The familiar route() decorator along with its methods optional argument can be used to declare the routes that handle the resource URLs exposed by the service. Working with JSON data is also simple, as JSON data included with a request is automatically exposed as a request.json Python dictionary and a response that needs to contain JSON can be easily generated from a Python dictionary using Flask’s jsonify() helper function. 

### Creating an API Blueprint

![main](../typical_error_message_of_api.png "main")

- One way to generate appropriate responses for all clients is to make the error handlers adapt their responses based on the format requested by the client,** a technique called content negotiation**

### User Authentication with Flask-HTTPAuth

- Web services, like regular web applications, need to protect information and ensure that it is not given to unauthorized parties. For this reason, RIAs must ask their users for login credentials and pass them to the server for verification.

- It was mentioned earlier that one of the characteristics of RESTful web services is that they are **stateless, which means that the server is not allowed to “remember” anything about the client between requests. Clients need to provide all the information necessary to carry out a request in the request itself, so all requests must include user credentials.**


- The current login functionality implemented with the help of Flask-Login stores data in the user session, which Flask stores by default in a client-side cookie, so the server does not store any user-related information; it asks the client to store it instead. It would appear that this implementation complies with the stateless requirement of REST, but the use of cookies in RESTful web services falls into a gray area, as it can be cumbersome for clients that are not web browsers to implement them. For that reason, it is generally seen as a bad design choice to use them.


- Because the RESTful architecture is based on the HTTP protocol, HTTP authentication is the preferred method used to send credentials, either in its Basic or Digest flavors. With HTTP authentication, user credentials are included in an Authorization header with all requests. 

- The HTTP authentication protocol is simple enough that it can be implemented directly, but the Flask-HTTPAuth extension provides a convenient wrapper that hides the protocol details in a decorator similar to Flask-Login’s login_required. 

pip install flask-httpauth

## Token Based Authentication

# P. 205


## Serializing Resources to and from JSON

- A frequent need when writing a web service is to convert internal representation of resources to and from JSON, which is the transport format used in HTTP requests and responses. 

- The url, author, and comments fields need to return the URLs for the respective resources, so these are generated with url\_for() calls to routes that will be defined in the API blueprint. Note that \_external=True is added to all url\_for() calls so that fully qualified URLs are returned instead of the relative URLs that are typically used within the context of a traditional web application

- The comment\_count field returns the number of comments that exist for the blog post. Although this is not a real attribute of the model, it is included in the resource representation as a convenience to the client

> from_json
- JSON dictionary. The body_html attribute is ignored since the server-side Markdown rendering is automatically triggered by a SQLAlchemy event whenever the body attribute is modified. The timestamp attribute does not need to be given, unless the client is allowed to backdate posts, which is not a feature of this application. The author field is not used because the client has no authority to select the author of a blog post; the only possible value for the author field is that of the authenticated user. The comments and comment\
_count attributes are automatically generated from a database relationship, so there is no useful information in them that is needed to create a model. Finally, the url field is ignored because in this implementation the resource URLs are defined by the server, not the client. 

- The errorhandler decorator is the same one that is used to register handlers for HTTP status codes, but in this usage it takes an Exception class as argument. The decorated function will be invoked any time an exception of the given class is raised. Note that the decorator is obtained from the API blueprint, so this handler will be invoked only when the exception is raised while a route from the blueprint is being processed. Using this technique, the code in view functions can be written very cleanly and concisely, without the need to include error checking. For example:


In [ ]:
@api.route('/posts/', methods=['POST']) 
def new_post():    
    post = Post.from_json(request.json)    
    post.author = g.current_user    
    db.session.add(post)    
    db.session.commit()    
    return jsonify(post.to_json())

## Implementing Resource Endpoints

# P. 209